In [ ]:
import os
import cv2
import shutil
import deeplabcut
import numpy as np
import math

In [ ]:
start_file  = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/ken_20210505_0939_XBI19_reversal_learning_foraging_SC3_DCS_frontcamera.mp4'

In [ ]:
dsv = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/ken_20210505_0939_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled.mp4' #deeplabcut.DownSampleVideo(start_file, width=320, height=240)

In [ ]:
video = cv2.VideoCapture(dsv)

# Get the frame rate and total number of frames
fps = int(video.get(cv2.CAP_PROP_FPS))
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

#*********************************************

target_br = 0.6
target_g = math.log(target_br)

#**********************************************

# Set the video writer
output_name = os.path.splitext(os.path.basename(dsv))[0].lstrip('.') + '_GC.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_name, fourcc, fps, (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))
start_frame = 0
# Set the current frame number to the start frame
video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
frame_number = 0
# Loop through the frames and write them to the output video
for i in range(start_frame, total_frames):
    ret, frame = video.read()
    if not ret:
        break

    frame_normalized = frame.astype(np.float32) / 255.0
    gray = cv2.cvtColor(frame_normalized, cv2.COLOR_BGR2GRAY)

    brightness = np.mean(frame_normalized)

    #finding and correcting gamma - a number is assigned to be the 'innate' gamma value of the frame, and that value is used to find the correction needed. 
    if brightness == 0:
        print('err')
        continue
    assigned_g = math.log(brightness)
    gamma = assigned_g/target_g

    # Normalize the pixel values to the range [0, 1]
    # frame_normalized = frame.astype(np.float32) / 255.0

    # Apply gamma correction
    frame_corrected = np.power(frame_normalized, 1/gamma)

    # Scale the pixel values back to the range [0, 255]
    frame_scaled = (frame_corrected * 255.0).astype(np.uint8)

    print(round(frame_number/total_frames*100, 2), end = '\r')
    frame_number += 1
    cv2.imshow('output', frame)
    # cv2.imshow('output', frame_scaled)
    out.write(frame_scaled)
# new = os.path.join(os.path.dirname(os.path.dirname(convid)), 'GC_Conv_vids', os.path.basename(convid))
# shutil.move(dsv, new)
print("Done - ", dsv)
print('')
# Release the video objects
video.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
os.mkdir("final_video")

In [ ]:
shutil.move('ken_20210505_0939_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled_GC.mp4', 'final_video/ken_20210505_0939_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled_GC.mp4')

In [ ]:
os.chdir('final_video')

In [ ]:
# dsv = deeplabcut.DownSampleVideo(start_file, width=320, height=240)

In [ ]:


file = 'ken_20210505_0939_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled_GC.mp4'
bodyparts = ['RightEye_Outer', 'RightEye_Top', 'RightEye_Bottom', 'RightEye_Inner', 'RightEye_Pupil', 'LeftEye_Outer', 'LeftEye_Top', 'LeftEye_Bottom', 'LeftEye_Inner', 'LeftEye_Pupil',
             'OutlineTop_Mid','RightNostrils_Top', 'RightNostrils_Bottom','LeftNostrils_Top', 'LeftNostrils_Bottom']
videotype = os.path.splitext(file)[-1].lstrip('.')  # or MOV, or avi, whatever you uploaded!
video_down = file

model_options = deeplabcut.create_project.modelzoo.Modeloptions
model_selection = 'primate_face'



In [ ]:
project_name = 'DLC_GazeXBI'
your_name = 'anc'

config_path, train_config_path = deeplabcut.create_pretrained_project(
    project_name,
    your_name,
    video_down,
    videotype=videotype,
    model=model_selection,
    analyzevideo=True,
    createlabeledvideo=False,
    copy_videos=True,
)



In [ ]:
edits = {
    'dotsize': 2,  # size of the dots!
    'pcutoff': 0.4,  # the higher, the more conservative the plotting!
}
deeplabcut.auxiliaryfunctions.edit_config(config_path, edits)

project_path = os.path.dirname(config_path)
full_video_path = os.path.join(
    project_path,
    'videos',
    os.path.basename(video_down),
)

# filter predictions (should already be done above ;):
deeplabcut.filterpredictions(config_path, full_video_path, videotype=videotype, )

# re-create the video with your edits!
# deeplabcut.CropVideo(config_path, full_video_path, 'crop', 

deeplabcut.create_labeled_video(config_path, full_video_path, videotype=videotype, displayedbodyparts=bodyparts, draw_skeleton = True, filtered=True)
